# Part 5 - Explore the Clusters
In this notebook, I will explore the clusters created by KMeans and create an interactive Folium map of the clusters.  

<div style="text-align: left">
    <h2>
        <a href="https://nbviewer.jupyter.org/github/KathrynDH/IBMCapstoneFinalProject/blob/master/Cluster%20Metro%20Areas.ipynb">&larr; Part 4</a>
    </h2>
</div>


In [4]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [5]:
#import libraries
import requests
import numpy as np
import pandas as pd
import ibm_boto3
import io

In [6]:
from IPython.display import Image 
from IPython.core.display import HTML 
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mpl

In [7]:
# The code was removed by Watson Studio for sharing.

In [8]:
#Import dataframe
obj = resource.Object(bucket_name=bucket, key='VenueDataWithClusters.json').get()
df = pd.read_json(io.BytesIO(obj['Body'].read()))
df.sort_index(inplace=True)
df.head()

,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
0,Tokyo,Japan,Asia,35.682839,139.759455,Café,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Lounge,Garden,French Restaurant,Dessert Shop,Thai Restaurant,Steakhouse,9
1,Delhi,India,Asia,28.651718,77.221939,Indian Restaurant,Bar,Café,Flea Market,Lounge,Asian Restaurant,Ice Cream Shop,South Indian Restaurant,Portuguese Restaurant,Plaza,5
2,Mexico City,Mexico,North America,19.432601,-99.133342,Mexican Restaurant,Art Museum,Museum,Arts & Crafts Store,Bar,Ice Cream Shop,Boutique,Bakery,Restaurant,Café,14
3,São Paulo,Brazil,South America,-23.550651,-46.633382,Brazilian Restaurant,Japanese Restaurant,Café,Bakery,Snack Place,Bookstore,Cosmetics Shop,Asian Restaurant,Art Gallery,Chinese Restaurant,9
4,Lagos,Nigeria,Africa,6.455057,3.394179,Lounge,African Restaurant,Bar,Café,Shopping Mall,Pizza Place,Hotel,Art Gallery,Italian Restaurant,Market,14


In [9]:
#How many metro areas are in each cluster?
df_count = df.loc[:,['Metropolitan','Cluster Labels']].groupby('Cluster Labels').count()
df_count

,Metropolitan
Cluster Labels,
0,1
1,12
2,1
3,1
4,21
5,8
6,1
7,6
8,1


In [10]:
#Look at the metro areas in each cluster
for i in df_count.index:
    print('Cluster ',i)
    with pd.option_context('display.max_rows', None):
        display(df[df['Cluster Labels']==i])

Cluster  0


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
30,Kinshasa,Democratic Republic of the Congo,Africa,-4.321706,15.312597,Café,Restaurant,Fast Food Restaurant,Lounge,Bakery,Steakhouse,Pool,Plaza,Pizza Place,Furniture / Home Store,0


Cluster  1


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
10,Chengdu,China,Asia,30.662420,104.063322,Chinese Restaurant,Noodle House,Tea Room,Department Store,Szechuan Restaurant,Hotpot Restaurant,Bookstore,Coffee Shop,Shopping Mall,Pedestrian Plaza,1
12,Shanghai,China,Asia,31.232274,121.469175,Chinese Restaurant,Lounge,Hotel Bar,Coffee Shop,Japanese Restaurant,Art Museum,Café,Xinjiang Restaurant,Multiplex,Park,1
23,Jakarta,Indonesia,Asia,-6.175394,106.827183,Park,Indonesian Restaurant,Asian Restaurant,Noodle House,Track,Soup Place,Soccer Field,Skate Park,Plaza,Performing Arts Venue,1
35,Rhine-Ruhr,Germany,Europe,51.437302,7.334511,Furniture / Home Store,Bakery,German Restaurant,Café,Ice Cream Shop,Supermarket,Big Box Store,Doner Restaurant,Arts & Crafts Store,Restaurant,1
48,Chūkyō (Nagoya),Japan,Asia,35.138700,136.966392,Ramen Restaurant,Japanese Restaurant,Park,Café,Bakery,BBQ Joint,Yoshoku Restaurant,Sake Bar,Chinese Restaurant,Restaurant,1
50,Bandung,Indonesia,Asia,-6.934469,107.604954,Museum,Noodle House,Park,Pet Store,Restaurant,None,None,None,None,None,1
52,Randstad,Netherlands,Europe,52.378042,5.226072,Sushi Restaurant,Restaurant,Supermarket,Gym,Gym / Fitness Center,Shopping Mall,Grocery Store,Park,Pet Store,Electronics Store,1
53,Busan,South Korea,Asia,35.179953,129.075236,Supermarket,Asian Restaurant,BBQ Joint,Coffee Shop,Concert Hall,Fast Food Restaurant,Gukbap Restaurant,Korean Restaurant,Multiplex,Noodle House,1
67,Manila,Philippines,Asia,14.590622,120.979970,Filipino Restaurant,Restaurant,Japanese Restaurant,Asian Restaurant,Bakery,Museum,Fast Food Restaurant,Italian Restaurant,Chinese Restaurant,Café,1
77,Berlin/Brandenburg,Germany,Europe,52.373411,13.511311,Greek Restaurant,Italian Restaurant,German Restaurant,Supermarket,Beach,Big Box Store,Ice Cream Shop,Park,Tea Room,River,1


Cluster  2


...

,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
6,Cairo,Egypt,Africa,30.048819,31.243666,Café,Lounge,Middle Eastern Restaurant,Italian Restaurant,Theater,Egyptian Restaurant,Lebanese Restaurant,Coffee Shop,Hotel Bar,Boat or Ferry,13
11,Dhaka,Bangladesh,Asia,23.759357,90.378814,Café,Fast Food Restaurant,Shopping Mall,Restaurant,Asian Restaurant,Burger Joint,Art Gallery,Coffee Shop,Department Store,Electronics Store,13
17,Istanbul,Turkey,Europe,41.009633,28.965165,Turkish Restaurant,Jewelry Store,Kebab Restaurant,Café,Restaurant,Garden,Art Gallery,Hotel,Seafood Restaurant,Coffee Shop,13
47,Bogotá,Colombia,South America,4.598080,-74.076044,Café,Restaurant,Italian Restaurant,History Museum,Burger Joint,Mexican Restaurant,Vegetarian / Vegan Restaurant,Argentinian Restaurant,South American Restaurant,Plaza,13
79,Munich,Germany,Europe,48.137108,11.575382,Café,Bavarian Restaurant,Restaurant,German Restaurant,Plaza,Cocktail Bar,Pizza Place,Boutique,Coffee Shop,Opera House,13
92,Sydney,Australia,Oceania,-33.854816,151.216454,Café,Australian Restaurant,Park,Pub,Theater,Scenic Lookout,Hotel Bar,Brewery,Pool,Coffee Shop,13
94,Alexandria,Egypt,Africa,31.199004,29.894378,Café,Seafood Restaurant,Ice Cream Shop,Coffee Shop,Sandwich Place,Dessert Shop,Pizza Place,Restaurant,Middle Eastern Restaurant,Sports Club,13
99,Khartoum,Sudan,Africa,15.593325,32.535650,Restaurant,Middle Eastern Restaurant,Burger Joint,African Restaurant,Hookah Bar,Pizza Place,Fast Food Restaurant,Café,Concert Hall,Asian Restaurant,13


Cluster  14


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
2,Mexico City,Mexico,North America,19.432601,-99.133342,Mexican Restaurant,Art Museum,Museum,Arts & Crafts Store,Bar,Ice Cream Shop,Boutique,Bakery,Restaurant,Café,14
4,Lagos,Nigeria,Africa,6.455057,3.394179,Lounge,African Restaurant,Bar,Café,Shopping Mall,Pizza Place,Hotel,Art Gallery,Italian Restaurant,Market,14
9,Los Angeles,United States,North America,34.053691,-118.242767,Bar,Coffee Shop,Sushi Restaurant,Mexican Restaurant,Plaza,Ice Cream Shop,Ramen Restaurant,Sandwich Place,Seafood Restaurant,Mediterranean Restaurant,14
25,Buenos Aires,Argentina,South America,-34.607562,-58.437076,Argentinian Restaurant,Gym,Bakery,Pizza Place,Ice Cream Shop,Restaurant,Café,Burger Joint,Bar,Plaza,14
36,Lahore,Pakistan,Asia,31.565608,74.314177,Pakistani Restaurant,Fast Food Restaurant,Ice Cream Shop,Restaurant,Market,Department Store,Italian Restaurant,BBQ Joint,Café,Park,14
40,Lima,Peru,South America,-12.062107,-77.036526,Park,Seafood Restaurant,Bar,Shopping Mall,Nightclub,Chinese Restaurant,Peruvian Restaurant,Plaza,Electronics Store,Pub,14
55,Dallas-Fort Worth,United States,North America,32.759907,-97.173563,Ice Cream Shop,American Restaurant,BBQ Joint,Bar,Park,Mexican Restaurant,Liquor Store,Grocery Store,Golf Course,Supermarket,14
66,Houston,United States,North America,29.758938,-95.367697,Bar,Cocktail Bar,Seafood Restaurant,Southern / Soul Food Restaurant,Park,Mexican Restaurant,New American Restaurant,Concert Hall,Lounge,Steakhouse,14
69,Miami,United States,North America,25.774266,-80.193659,Italian Restaurant,Cocktail Bar,Seafood Restaurant,Gym,Lounge,Bar,Coffee Shop,Peruvian Restaurant,Pool,American Restaurant,14
71,Atlanta,United States,North America,33.749099,-84.390185,Art Gallery,Lounge,Mexican Restaurant,Café,Coffee Shop,Indian Restaurant,Performing Arts Venue,Paper / Office Supplies Store,Korean Restaurant,Mediterranean Restaurant,14


Cluster  15


,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
13,Chongqing,China,Asia,29.558571,106.549282,Shopping Mall,Coffee Shop,Chinese Restaurant,Electronics Store,Park,Nightclub,Asian Restaurant,Museum,Scenic Lookout,Public Art,15
18,Bangkok,Thailand,Asia,13.753893,100.816080,Noodle House,Thai Restaurant,Café,Asian Restaurant,Coffee Shop,Convenience Store,Flea Market,Shopping Mall,Park,Supermarket,15
33,Nanjing,China,Asia,32.060974,118.791646,Shopping Mall,Chinese Restaurant,Nightclub,Café,Indian Restaurant,Japanese Restaurant,Multiplex,Electronics Store,Park,Jiangsu Restaurant,15
44,Johannesburg,South Africa,Africa,-26.205000,28.049722,Coffee Shop,Café,Art Gallery,Portuguese Restaurant,Shopping Mall,Flea Market,Asian Restaurant,Bar,Theater,Indian Restaurant,15
45,Guangzhou,China,Asia,23.130196,113.259294,Chinese Restaurant,Nightclub,Shopping Mall,History Museum,Asian Restaurant,Park,Movie Theater,Cantonese Restaurant,Dessert Shop,Clothing Store,15
46,Qingdao,China,Asia,36.063426,120.378558,Shopping Mall,Bar,Coffee Shop,Park,Beach,Café,Chinese Restaurant,Dumpling Restaurant,Museum,Market,15
89,Shenzhen,China,Asia,22.544570,114.054535,Shopping Mall,Café,Chinese Restaurant,Nightclub,Japanese Restaurant,Lounge,Italian Restaurant,Turkish Restaurant,Thai Restaurant,Park,15


In [13]:
#Create an interactive folium map with different color markers for each cluster
map_clusters = folium.Map(location=[0,0], zoom_start=2)

#set color scheme for the clusters
map_colors = [colors.rgb2hex(cm.tab20b(i)) for i in range(len(df_count.index)+1)]

# add markers to the map
markers_colors = []
for latitude, longitude, metro, country, cluster in zip(df['Latitude'], df['Longitude'], df['Metropolitan'], df['Country'], df['Cluster Labels']):
    label = folium.Popup(str(metro) + ', ' + str(country)+'- Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color=map_colors[cluster-1],
        fill=True,
        fill_color=map_colors[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save('map.html')
map_clusters

### View the interactive map
https://kathryndh.github.io/clustermap.html

In [12]:
#Save the map as an html file
obj = io.open('map.html','rb')
resource.Bucket(name=bucket).put_object(Key='map.html', Body=obj.read())
obj.close()
print('Saved')

Saved


<div style="text-align: left">
    <h2>
        <a href="https://nbviewer.jupyter.org/github/KathrynDH/IBMCapstoneFinalProject/blob/master/Cluster%20Metro%20Areas.ipynb">&larr; Part 4</a>
    </h2>
</div>